#### Required Library to be installed

In [1]:
# !pip3 install tensorly --quiet
# !pip3 install tensorflow --quiet
# !pip3 install openpyxl --quiet
# !pip3 install statsmodels --quiet
# !pip3 install matplotlib --quiet
# !pip3 install seaborn
# !pip3 install plotly==5.11.0 --quiet
# !pip3 install --upgrade nbformat --quiet
# !pip3 install ipykernel
# !pip3 install -U kaleido
# !pip3 install pandas numpy --quiet
import warnings
warnings.filterwarnings('ignore')


#### Import Required libraries

In [2]:
import pandas as pd
import numpy as np
import tensorly as tl
import tensorflow as tf
from tensorly.decomposition import non_negative_tucker as tucker
from tensorly.decomposition import non_negative_tucker_hals as tucker_hal
# import seaborn as sns
# import matplotlib.pyplot as plt
from tensorly.metrics.regression import RMSE
import plotly.graph_objects as go
# from matplotlib import rcParams
import plotly.express as px
# rcParams['figure.figsize'] = 20,6

#### Read final Data

In [3]:
tensor_data = pd.read_csv("data/FINAL_DATA.csv")
tensor_data = tensor_data.iloc[:,1:]

In [4]:
stages = ['AKTARAJA_CIR','AMT_COURT','ANJANEY_CM','ARALIMARA','AVARAGERE','AVARGOLA','AVK_CLG','BADA_CR','BAGIRATA_CL','BATHI','BSNL','CG_HOSPITAL',
'D_COURT','DAVANAGERE','DC_OFFICE','DCM_LAYOUT','GANESH_BH','GANESH_TEMP','GMIT_CLG','HARIHARA','HONDA_CL','ITI','JAYANAGAR','JHPATEL_NAGAR','KAKKARGOLA',
'KENDRIYA_VI','KONDAJJI','KSRTC_BUS_ST','KUNDWADA','LALITKALA_CLG','LAXMI_FL_MILL','LIC','MEDICAL_CLG','NINCHANA_SL','NITUVALLI_PL','NUTHAN_CLG','OLD_RTO',
'PRIVATE_BS_ST','RAILWAY_ST','RANEBENNUR','RTO_OF','SARSWATI_NAGAR','SHAMNUR','SOG_COLONY','SS_HOSPITAL','THOLHUNSE','UNIVERSITY','VIDYANAGARA',
'VIDYARTI_BH','VINOBNAGAR','YARGUNTE']

In [5]:
len(stages)

51

#### Tensor Decomposition

- Remove the duplicates

In [5]:
def remove_same_od_count(df,time):
    df.loc[df['Origin'] == df['Destination'], 'Trip_Count'] = 0
    if(time=='Hour'):
        df['Trip_Count'] = df['Trip_Count']/(30*6+3)
    if(time == 'DAILY'):
        df['Trip_Count'] = df['Trip_Count']/(4*4 + 5*2)
    return df

- Prepare the tensor for input in the Tucker Algorithm

In [6]:
def NTD_data_prepare(tensor_data , time):
    month = tensor_data[[time,'ORIGIN','DESTINATION']]
    grp_month = month.groupby([time,'ORIGIN','DESTINATION'])
    tensor_list_month = []
    for it , df in grp_month:
        tensor_list_month.append([it[0], it[1] , it[2], len(df)])
    monthly = pd.DataFrame(tensor_list_month , columns=[time,'Origin','Destination','Trip_Count'])
    
    #Remove same origin-destination count
    monthly_data = remove_same_od_count(monthly,time)
    
    month_grp = monthly_data.groupby([time])
    month_tensor_data = []

    for i,df in month_grp:
        pivot_table_for_month = df.pivot_table(index='Origin',columns='Destination',values='Trip_Count').fillna(0)
        month_tensor_data.append(tl.tensor(pivot_table_for_month))
    ten = tf.convert_to_tensor(month_tensor_data)
    return ten

- Function to Export the factor matrices to csv 

In [7]:
def get_Core_factor(factor,name1,name2,name3):
    fc_0 = pd.DataFrame(factor[0])
    fc_1 = pd.DataFrame(factor[1])
    fc_2 = pd.DataFrame(factor[2])
    fc_0.to_csv(name1)
    fc_1.to_csv(name2)
    fc_2.to_csv(name3)

- Function to get the optimized rank for the Tucker Algorithm based on RMSE score

In [8]:
def optimised_rank(data):
    rank = []
    rmse = []
    for r1 in range(2,6):
        for r2 in range(2,10):
            for r3 in range(2,10):
                core1 , error = tucker(data,rank=[r1,r2,r3],return_errors=True)
                tucker_reconstruction_mu = tl.tucker_to_tensor(core1)
                er = RMSE(data, tucker_reconstruction_mu)
                rmse.append(er)
                rank.append( str(r1)+','+str(r2)+','+str(r3))
    
    dc = {'Rank':rank , 'RMSE':rmse}
    h = pd.DataFrame(dc)
    h = h.sort_values('RMSE',ascending=False)
    return h

In [9]:
# def optimised_rank_version_2(data):
#     rank = []
#     rmse = []
#     for r1 in range(2,6):
#         for r2 in range(5,52):
#             for r3 in range(5,52):
#                 core1 , error = tucker(data,rank=[r1,r2,r3],return_errors=True)
#                 tucker_reconstruction_mu = tl.tucker_to_tensor(core1)
#                 er = RMSE(data, tucker_reconstruction_mu)
#                 rmse.append(er)
#                 rank.append( str(r1)+','+str(r2)+','+str(r3))
    
#     dc = {'Rank':rank , 'RMSE':rmse}
#     h = pd.DataFrame(dc)
#     h = h.sort_values('RMSE',ascending=False)
#     return h

#### Get the core and factor

#### Hourly ( rank = [3,9,9] )

In [9]:
# tensor for input
tensor_final_1 = NTD_data_prepare(tensor_data,'Hour')

In [10]:
len(tensor_final_1)

14

- rank and RMSE score df

In [11]:
h = optimised_rank(tensor_final_1)
h.to_csv('data_final/Rank_hourly.csv')

In [12]:
h = h[h['RMSE']>=1.186063]

- Plot Variation in rank vs RMSE score

In [13]:
#scale = (width_in_mm / 25.4) / (width_default_px / dpi) = (130 / 25.4) / (700 / 300) = 2.1934758155230596
fig = px.line(h, 
                 x='Rank', 
                 y='RMSE', 
                 width=1400,
                 height=500
                 )
fig.update_layout(
    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_traces(marker_size=5)
fig.show()
fig.write_image('graphs/hr rank vs rmse.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr rank vs rmse.pdf',scale=(150 / 25.4) / (700 / 300))

In [15]:
# h2 = optimised_rank_version_2(tensor_final_1)

In [16]:
# fig = px.line(h2, 
#                  x='Rank', 
#                  y='RMSE', 
#                  title='Hourly Rank vs RMSE score',
#                  width=1400,
#                  height=500
#                  )
# fig.update_layout(
#     font_family='Arial Black',
#     font_size=15
# )
# fig.update(layout=dict(title=dict(x=0.5)))
# fig.update_traces(marker_size=5)
# fig.show()
# fig.write_image('hr rank vs rmse.jpeg',scale=(150 / 25.4) / (700 / 900))
# fig.write_image('hr rank vs rmse.pdf',scale=(150 / 25.4) / (700 / 300))

- core and factor

In [14]:
core1 , factor1 = tucker(tensor_final_1,rank=[3,9,9],normalize_factors=True)

#### Graph for factors


In [15]:
factor1_0 = pd.DataFrame(factor1[0])
factor1_0.reset_index(inplace=True) 
factor1_0['index'] = factor1_0['index'] + 7
factor1_0.columns = ['Hours','H1','H2','H3']

In [16]:
fig = px.line(factor1_0,
        x=[7,8,9,10,11,12,13,14,15,16,17,18,19,20],
        y=['H1','H2','H3'],
        markers=True
        )
fig.update_layout(
    # title='Hourly Temporal Pattern',
    xaxis_title = 'Time (in Hour)',
    yaxis_title = 'Factor Values',
    legend_title = 'Patterns',
    font_family="Arial",
    font_size=15,
    width=1400,
    height=500
)
fig.update_layout(
    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/hr_temp_ptn_line.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_temp_ptn_line.pdf',scale=(150 / 25.4) / (700 / 300))

In [17]:
fig = px.imshow(factor1[0],aspect="auto",
                labels=dict(y="Hours", x="Patterns", color="Factor"),
                y=[7,8,9,10,11,12,13,14,15,16,17,18,19,20],
                x=['H1','H2','H3'],
                #title="Hourly Temporal Pattern",
                width=1200,
                height=700,
                # color_continuous_scale='aggrnyl',
                # color_continuous_scale='aggrnyl_r'
                )

fig.update_layout(
    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))

fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/hr_temp_ptn_heatmap_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_temp_ptn_heatmap_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [18]:
fig = px.imshow(factor1[0],aspect="auto",
                labels=dict(y="Hours", x="Patterns", color="Factor"),
                y=[7,8,9,10,11,12,13,14,15,16,17,18,19,20],
                x=['H1','H2','H3'],
               # title="Hourly Temporal Pattern",
                width=1200,
                height=700,
                color_continuous_scale='aggrnyl'
                )

fig.update_layout(
    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))

fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/hr_temp_ptn_heatmap_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_temp_ptn_heatmap_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [19]:
fig = px.imshow(factor1[0],aspect="auto",
                labels=dict(y="Hours", x="Patterns", color="Factor"),
                y=[7,8,9,10,11,12,13,14,15,16,17,18,19,20],
                x=['H1','H2','H3'],
               # title="Hourly Temporal Pattern",
                width=1200,
                height=700,
                color_continuous_scale='aggrnyl_r'
                )

fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))

fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/hr_temp_ptn_heatmap_2_r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_temp_ptn_heatmap_2_r.pdf',scale=(150 / 25.4) / (700 / 300))

In [23]:
# fig = px.imshow(factor1[1],aspect="auto",
#                 labels=dict(x="Patterns", y="Origin", color="Factors"),
#                 x=[1, 2, 3,4,5,6,7,8,9],
#                 y=stages,
#                 title="Hourly Origin Spatial Pattern",
#                 width=1200,
#                 height=1200,
#                 # color_continuous_scale='aggrnyl_r'
#                 )

# fig.update_layout(
#     font_family='Arial Black',
#     font_size=15
# )
# fig.update(layout=dict(title=dict(x=0.5)))
# fig.update_xaxes(side="bottom")
# fig.show()

In [24]:
# fig = px.imshow(factor1[2],aspect="auto",
#                 labels=dict(x="Patterns", y="Destination", color="Factors"),
#                 x=[1, 2, 3,4,5,6,7,8,9],
#                 y=stages,
#                 title="Hourly Destination Spatial Pattern ",
#                 width=1200,
#                 height=1200,
#                 # color_continuous_scale='aggrnyl_r'
#                 )

# fig.update_layout(
#     font_family='Arial Black',
#     font_size=15
# )
# fig.update(layout=dict(title=dict(x=0.5)))
# fig.update_xaxes(side="bottom")
# fig.show()

#### Daily ( rank => [ 3,9,9 ]  )

In [20]:
tensor_final_2 = NTD_data_prepare(tensor_data,'DAILY')

In [21]:
d = optimised_rank(tensor_final_2)
d.to_csv('data_final/Rank_Daily.csv')
d = d.drop(63)

In [22]:
fig = px.line(d, 
                 x='Rank', 
                 y='RMSE', 
                 width=1400,
                 height=500
                 )
fig.update_layout(
    font_family='Arial',
    font_size=15,
    xaxis = dict(
        tickmode ='array',
    tickvals = ['3,5,2','2,6,2','2,3,7','5,4,3','5,3,8','4,4,5','5,9,5','5,6,9','2,9,6','3,8,7','3,9,9']
    )

)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_traces(marker_size=5)
fig.show()
fig.write_image('graphs/d rank vs rmse.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d rank vs rmse.pdf',scale=(150 / 25.4) / (700 / 300))

In [28]:
# d2 = optimised_rank_version_2(tensor_final_2)
# fig = px.line(d2, 
#                  x='Rank', 
#                  y='RMSE', 
#                  title='Daily Rank vs RMSE score',
#                  width=1400,
#                  height=600
#                  )
# fig.update_layout(
#     font_family='Arial Black',
#     font_size=7,
#     title_font_size=15,

# )
# fig.update(layout=dict(title=dict(x=0.5)))
# fig.update_traces(marker_size=5)
# fig.show()
# fig.write_image('d rank vs rmse.jpeg',scale=(150 / 25.4) / (700 / 900))
# fig.write_image('d rank vs rmse.pdf',scale=(150 / 25.4) / (700 / 300))

In [23]:
core2 , factor2 = tucker(tensor_final_2,rank=[3,9,9],normalize_factors=True)

In [24]:
factor2_0 = pd.DataFrame(factor2[0])
factor2_0.reset_index(inplace=True)
factor2_0.columns = ['Days','D1','D2','D3'] 

In [25]:
fig = px.line(factor2_0,
        x=['SUN','MON','TUE','WED','THU','FRI','SAT'],
        y=['D1','D2','D3'],
        markers=True
        )

fig.update_layout(
   # title='Daily Temporal Pattern',
    xaxis_title = 'Days',
    yaxis_title = 'Factor Values',
    legend_title = 'Patterns',
    font_family="Arial",

    font_size=15,
    width=1400,
    height=600
)
fig.update(layout=dict(title=dict(x=0.5)))

fig.update_xaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/d_temp_ptn_line.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_temp_ptn_line.pdf',scale=(150 / 25.4) / (700 / 300))

In [26]:
fig = px.imshow(factor2[0],aspect="auto",
                labels=dict(y="Days", x="Patterns", color="Factor"),
                y=['SUN','MON','TUE','WED','THU','FRI','SAT'],
                x=['D1','D2','D3'],
               # title="Daily Temporal Pattern",
                width=1200,
                height=700,
                # color_continuous_scale='aggrnyl_r'
                )
fig.update_layout(

    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))                
fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/d_temp_ptn_heatmap_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_temp_ptn_heatmap_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [27]:
fig = px.imshow(factor2[0],aspect="auto",
                labels=dict(y="Days", x="Patterns", color="Factor"),
                y=['SUN','MON','TUE','WED','THU','FRI','SAT'],
                x=['D1','D2','D3'],
               # title="Daily Temporal Pattern",
                width=1200,
                height=700,
                color_continuous_scale='aggrnyl'
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))                
fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/d_temp_ptn_heatmap_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_temp_ptn_heatmap_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [28]:
fig = px.imshow(factor2[0],aspect="auto",
                labels=dict(y="Days", x="Patterns", color="Factor"),
                y=['SUN','MON','TUE','WED','THU','FRI','SAT'],
                x=['D1','D2','D3'],
               # title="Daily Temporal Pattern",
                width=1200,
                height=700,
                color_continuous_scale='aggrnyl_r'
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))                
fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/d_temp_ptn_heatmap_2_r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_temp_ptn_heatmap_2_r.pdf',scale=(150 / 25.4) / (700 / 300))

In [29]:
# fig = px.imshow(factor2[1],aspect="auto",
#                 labels=dict(x="PATTERN", y="ORIGIN", color="FACTORS"),
#                 x=[1, 2, 3,4,5,6,7,8,9],
#                 y=stages,
#                 title="Daily Origin Spatial Pattern",
#                 width=1200,
#                 height=1200
#                 )
# fig.update_xaxes(side="bottom")
# fig.show()

In [30]:
# fig = px.imshow(factor2[2],aspect="auto",
#                 labels=dict(x="PATTERN", y="DESTINATION", color="FACTORS"),
#                 x=[1, 2, 3,4,5,6,7,8,9],
#                 y=stages,
#                 title="Daily Destination Spatial Pattern ",
#                 width=1200,
#                 height=1200
#                 )
# fig.update_xaxes(side="bottom")
# fig.show()

#### Monthly rank => ( [ 5,9,9 ] )

In [31]:
tensor_final_3 = NTD_data_prepare(tensor_data,'MONTHLY')

In [32]:
m = optimised_rank(tensor_final_3)
m.to_csv('data_final/Rank_Monthly.csv')
# m[m['Rank']=='5,9,9']

In [33]:
fig = px.line(m, 
                 x='Rank', 
                 y='RMSE', 
                 width=1400,
                 height=500
                 )
fig.update_layout(
    font_family='Arial',
    font_size=12
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_traces(marker_size=5)
fig.show()
fig.write_image('graphs/m rank vs rmse.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m rank vs rmse.pdf',scale=(150 / 25.4) / (700 / 300))

In [40]:
# m3 = optimised_rank_version_2(tensor_final_3)

In [41]:
# fig = px.line(m3, 
#                  x='Rank', 
#                  y='RMSE', 
#                  title='Daily Rank vs RMSE score',
#                  width=1400,
#                  height=600
#                  )
# fig.update_layout(
#     font_family='Arial Black',
#     font_size=7,
#     title_font_size=15,

# )
# fig.update(layout=dict(title=dict(x=0.5)))
# fig.update_traces(marker_size=5)
# fig.show()
# fig.write_image('m rank vs rmse.jpeg',scale=(150 / 25.4) / (700 / 900))
# fig.write_image('m rank vs rmse.pdf',scale=(150 / 25.4) / (700 / 300))

In [34]:
core3 , factor3 = tucker(tensor_final_3,rank=[5,9,9],normalize_factors=True)

In [35]:
factor3_0 = pd.DataFrame(factor3[0])
factor3_0.reset_index(inplace=True)
factor3_0['index'] =  ['April','May','June','July','Aug','Sept']
factor3_0.columns = ['Months' , 'M1','M2','M3','M4','M5']

In [36]:
fig = px.line(factor3_0,
        x=['April','May','June','July','Aug','Sept'],
        y=['M1','M2','M3','M4','M5'],
        markers=True
        )
        
fig.update_layout(
   # title='Monthly Temporal Pattern',
    xaxis_title = 'Months',
    yaxis_title = 'Factor Values',
    legend_title = 'Patterns',
    font_family="Arial",

    font_size=15,
    width=1400,
    height=600
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/m_temp_ptn_line.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_temp_ptn_line.pdf',scale=(150 / 25.4) / (700 / 300))

In [37]:
fig = px.imshow(factor3[0],aspect="auto",
                labels=dict(y="Months", x="Patterns", color="Factor"),
                y=['April','May','June','July','Aug','Sept'],
                x=['M1','M2','M3','M4','M5'],
               # title="Monthly Temporal Pattern",
                width=1200,
                height=700,
                #  color_continuous_scale='aggrnyl_r'
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/m_temp_ptn_heatmap_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_temp_ptn_heatmap_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [38]:
fig = px.imshow(factor3[0],aspect="auto",
                labels=dict(y="Months", x="Patterns", color="Factor"),
                y=['April','May','June','July','Aug','Sept'],
                x=['M1','M2','M3','M4','M5'],
               # title="Monthly Temporal Pattern",
                width=1200,
                height=700,
                 color_continuous_scale='aggrnyl'
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/m_temp_ptn_heatmap_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_temp_ptn_heatmap_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [39]:
fig = px.imshow(factor3[0],aspect="auto",
                labels=dict(y="Months", x="Patterns", color="Factor"),
                y=['April','May','June','July','Aug','Sept'],
                x=['M1','M2','M3','M4','M5'],
               # title="Monthly Temporal Pattern",
                width=1200,
                height=700,
                 color_continuous_scale='aggrnyl_r'
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('graphs/m_temp_ptn_heatmap_2_r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_temp_ptn_heatmap_2_r.pdf',scale=(150 / 25.4) / (700 / 300))

In [61]:
fig = px.imshow(factor3[0],aspect="auto",
                labels=dict(y="Months", x="Patterns", color="Factor"),
                y=['April','May','June','July','Aug','Sept'],
                x=['M1','M2','M3','M4','M5'],
               # title="Monthly Temporal Pattern",
                width=350,
                height=270,
                 color_continuous_scale='aggrnyl_r'
                )
fig.update_layout(
    font_family='Arial',

    # font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom",tickmode='linear')
fig.update_yaxes(tickmode='linear')
fig.show()
fig.write_image('report/m_temp_ptn_heatmap.jpeg',scale=(150 / 25.4) / (700 / 900))
#fig.write_image('report/m_temp_ptn_heatmap.pdf',scale=(150 / 25.4) / (700 / 300))

In [48]:
#Bar graphs
fig = px.bar(factor3_0 , x='Months' , y=['M1','M2','M3','M4','M5'],barmode='group',
#title='Monthly Temporal Pattern',
width=1400 , height=600 )
fig.update_layout(
    font_family='Arial',

    font_size=15,
    legend=dict(title="Patterns")
)
fig.update_yaxes(title='Factor values')
fig.update(layout=dict(title=dict(x=0.5)))
fig.show()
fig.write_image('graphs/m_temp_ptn_bar.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_temp_ptn_bar.pdf',scale=(150 / 25.4) / (700 / 300))

In [49]:
# fig = px.imshow(factor3[1],aspect="auto",
#                 labels=dict(x="PATTERN", y="ORIGIN", color="FACTORS"),
#                 x=[1, 2, 3,4,5,6,7,8,9],
#                 y=stages,
#                 title="Monthly Origin Spatial Pattern",
#                 width=1200,
#                 height=1200
#                 )
# fig.update_xaxes(side="bottom")
# fig.show()

In [50]:
# fig = px.imshow(factor3[2],aspect="auto",
#                 labels=dict(x="PATTERN", y="DESTINATION", color="FACTORS"),
#                 x=[1, 2, 3,4,5,6,7,8,9],
#                 y=stages,
#                 title="Monthly Destination Spatial Pattern ",
#                 width=1200,
#                 height=1200
#                 )
# fig.update_xaxes(side="bottom")
# fig.show()

### CORE TENSOR PLOT HEATMAPS

#### Hourly

In [51]:
fig = px.imshow(core1[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="H1 Pattern",
                width=1400,
                height=700
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/hr_st_ptn_H1_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_st_ptn_H1_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [52]:
fig = px.imshow(core1[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="H1 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/hr_st_ptn_H1_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_st_ptn_H1_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [53]:
fig = px.imshow(core1[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="H1 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/hr_st_ptn_H1_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_st_ptn_H1_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [54]:
fig = px.imshow(core1[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="H2 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/hr_st_ptn_H2_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_st_ptn_H2_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [55]:
fig = px.imshow(core1[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="H2 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(

    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/hr_st_ptn_H2_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_st_ptn_H2_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [56]:
fig = px.imshow(core1[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="H2 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(

    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/hr_st_ptn_H2_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_st_ptn_H2_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [57]:
fig = px.imshow(core1[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="H3 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/hr_st_ptn_H3_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_st_ptn_H3_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [58]:
fig = px.imshow(core1[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="H3 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/hr_st_ptn_H3_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_st_ptn_H3_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [59]:
fig = px.imshow(core1[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="H3 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/hr_st_ptn_H3_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/hr_st_ptn_H3_2r.pdf',scale=(150 / 25.4) / (700 / 300))

#### Daily

In [60]:
fig = px.imshow(core2[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="D1 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/d_st_ptn_D1_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_st_ptn_D1_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [61]:
fig = px.imshow(core2[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="D1 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/d_st_ptn_D1_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_st_ptn_D1_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [62]:
fig = px.imshow(core2[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="D1 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(

    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/d_st_ptn_D1_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_st_ptn_D1_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [63]:
fig = px.imshow(core2[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="D2 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/d_st_ptn_D2_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_st_ptn_D2_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [64]:
fig = px.imshow(core2[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="D2 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(

    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/d_st_ptn_D2_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_st_ptn_D2_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [65]:
fig = px.imshow(core2[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="D2 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/d_st_ptn_D2_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_st_ptn_D2_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [66]:
fig = px.imshow(core2[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="D3 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/d_st_ptn_D3_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_st_ptn_D3_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [67]:
fig = px.imshow(core2[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="D3 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/d_st_ptn_D3_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_st_ptn_D3_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [68]:
fig = px.imshow(core2[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="D3 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/d_st_ptn_D3_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/d_st_ptn_D3_2r.pdf',scale=(150 / 25.4) / (700 / 300))

#### Monthly

In [69]:
fig = px.imshow(core3[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M1 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M1_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M1_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [70]:
fig = px.imshow(core3[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M1 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M1_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M1_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [71]:
fig = px.imshow(core3[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M1 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M1_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M1_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [55]:





fig = px.imshow(core3[0],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M1 Pattern",
                width=350,
                height=270,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial'
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('report/m_st_ptn_M1.jpeg',scale=(150 / 25.4) / (700 / 900))
# fig.write_image('graphs/m_st_ptn_M1_2r.pdf',scale=(150 / 25.4) / (700 / 300))









In [72]:
fig = px.imshow(core3[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M2 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M2_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M2_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [73]:
fig = px.imshow(core3[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M2 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M2_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M2_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [74]:
fig = px.imshow(core3[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M2 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M2_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M2_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [56]:



fig = px.imshow(core3[1],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M2 Pattern",
                width=350,
                height=270,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial'
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('report/m_st_ptn_M2.jpeg',scale=(150 / 25.4) / (700 / 900))
# fig.write_image('graphs/m_st_ptn_M2_2r.pdf',scale=(150 / 25.4) / (700 / 300))





In [75]:
fig = px.imshow(core3[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M3 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M3_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M3_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [76]:
fig = px.imshow(core3[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M3 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M3_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M3_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [77]:
fig = px.imshow(core3[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M3 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M3_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M3_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [57]:




fig = px.imshow(core3[2],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M3 Pattern",
                width=350,
                height=270,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial'
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('report/m_st_ptn_M3.jpeg',scale=(150 / 25.4) / (700 / 900))
# fig.write_image('graphs/m_st_ptn_M3_2r.pdf',scale=(150 / 25.4) / (700 / 300))





In [78]:
fig = px.imshow(core3[3],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M4 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M4_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M4_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [79]:
fig = px.imshow(core3[3],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M4 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M4_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M4_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [80]:
fig = px.imshow(core3[3],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M4 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M4_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M4_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [58]:






fig = px.imshow(core3[3],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M4 Pattern",
                width=350,
                height=270,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial'
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('report/m_st_ptn_M4_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
# fig.write_image('graphs/m_st_ptn_M4_2r.pdf',scale=(150 / 25.4) / (700 / 300))







In [81]:
fig = px.imshow(core3[4],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M5 Pattern",
                width=1400,
                height=700,
                # color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M5_1.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M5_1.pdf',scale=(150 / 25.4) / (700 / 300))

In [82]:
fig = px.imshow(core3[4],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M5 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl',
                )
fig.update_layout(

    font_family='Arial',
    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M5_2.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M5_2.pdf',scale=(150 / 25.4) / (700 / 300))

In [83]:
fig = px.imshow(core3[4],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M5 Pattern",
                width=1400,
                height=700,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial',

    font_size=15
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('graphs/m_st_ptn_M5_2r.jpeg',scale=(150 / 25.4) / (700 / 900))
fig.write_image('graphs/m_st_ptn_M5_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [60]:






fig = px.imshow(core3[4],aspect="auto",
                labels=dict(x="Destination Pattern", y="Origin Pattern", color="value"),
                x=['D1','D2','D3','D4','D5','D6','D7','D8','D9'],
                y=['O1','O2','O3','O4','O5','O6','O7','O8','O9'],
                title="M5 Pattern",
                width=350,
                height=270,
                color_continuous_scale='aggrnyl_r',
                )
fig.update_layout(
    font_family='Arial'
)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_xaxes(side="bottom")
fig.show()
fig.write_image('report/m_st_ptn_M5.jpeg',scale=(150 / 25.4) / (700 / 900))
# fig.write_image('graphs/m_st_ptn_M5_2r.pdf',scale=(150 / 25.4) / (700 / 300))

In [ ]:
# pd.DataFrame(core1[0]).to_csv('data_final/core_hr_1.csv')
# pd.DataFrame(core1[1]).to_csv('data_final/core_hr_2.csv')
# pd.DataFrame(core1[2]).to_csv('data_final/core_hr_3.csv')
# pd.DataFrame(core2[0]).to_csv('data_final/core_daily_1.csv')
# pd.DataFrame(core2[1]).to_csv('data_final/core_daily_2.csv')
# pd.DataFrame(core2[2]).to_csv('data_final/core_daily_3.csv')
# pd.DataFrame(core3[0]).to_csv('data_final/core_month_1.csv')
# pd.DataFrame(core3[1]).to_csv('data_final/core_month_2.csv')
# pd.DataFrame(core3[2]).to_csv('data_final/core_month_3.csv')
# pd.DataFrame(core3[3]).to_csv('data_final/core_month_4.csv')
# pd.DataFrame(core3[4]).to_csv('data_final/core_month_5.csv')

# THE END